In [31]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import CRS, Transformer
import os
import matplotlib.colors as mcolors

In [22]:
#ACQUISITION

#All coral data files are from MAREANO
gml_file_coral_reefs_trondelag = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml"
gdf_coral_reefs_trondelag = gpd.read_file(gml_file_coral_reefs_trondelag)

gml_file_coral_reefs_more_og_romsdal = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml"
gdf_coral_reefs_more_og_roms = gpd.read_file(gml_file_coral_reefs_more_og_romsdal)

gml_file_coral_reefs_nordland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml"
gdf_coral_reefs_nordland = gpd.read_file(gml_file_coral_reefs_nordland)

gml_file_coral_reefs_vestland = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml"
gdf_coral_reefs_vestland = gpd.read_file(gml_file_coral_reefs_vestland)

gml_file_coral_reefs_troms = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml"
gdf_coral_reefs_troms = gpd.read_file(gml_file_coral_reefs_troms)

gml_file_coral_reefs_finnmark = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml"
gdf_coral_reefs_finnmark = gpd.read_file(gml_file_coral_reefs_finnmark)

gml_file_coral_reefs_norskehavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml"
gdf_coral_reefs_norskehavet = gpd.read_file(gml_file_coral_reefs_norskehavet)

gml_file_coral_reefs_barentshavet = "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
gdf_coral_reefs_barentshavet = gpd.read_file(gml_file_coral_reefs_barentshavet)


#Define file paths to merge
gml_files = {
    "Trondelag": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_50_Trondelag_25832_Korallrev_GML.gml",
    "More_og_Romsdal": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_15_More_og_Romsdal_25832_Korallrev_GML.gml",
    "Nordland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_18_Nordland_25833_Korallrev_GML.gml",
    "Vestland": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_46_Vestland_25832_Korallrev_GML.gml",
    "Troms": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_55_Troms_25833_Korallrev_GML.gml",
    "Finnmark": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_56_Finnmark_25833_Korallrev_GML.gml",
    "Norskehavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_63_Norskehavet_25832_Korallrev_GML.gml",
    "Barentshavet": "/cluster/home/maikents/coral-mapping/raw_data/KystOgFiskeri_64_Barentshavet_vest_25833_Korallrev_GML.gml"
}

#Read all files into a list of GeoDataFrames
gdfs = [gpd.read_file(file).to_crs("EPSG:4326") for file in gml_files.values()]

#Combine
gdf_all_coral_files = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))


In [28]:
#CLEANING
#remove any entries with nan values for longitude or latitude (none of the points used here had this)
gdf_all_coral_files = gdf_all_coral_files.dropna(subset=['lengdegrad', 'breddegrad'])

In [ ]:
#TRANSFORMATION

#first we need to obtain the SINMOD crs (coordinate reference system)

def obtain_sinmod_crs(sinmod_data):
    
    grid_mapping = sinmod_data.variables['grid_mapping']
    horizontal_resolution = grid_mapping.getncattr('horizontal_resolution')

    crs_sinmod = CRS.from_proj4(f"+proj=stere "
                                f"+lat_0={grid_mapping.latitude_of_projection_origin} "
                                f"+lat_ts={grid_mapping.standard_parallel} "
                                f"+lon_0={grid_mapping.straight_vertical_longitude_from_pole} "
                                f"+x_0={grid_mapping.false_easting} "
                                f"+y_0={grid_mapping.false_northing} "
                                f"+a={grid_mapping.semi_major_axis} "
                                f"+b={grid_mapping.semi_minor_axis} "
                                f"+units=m +no_defs")
    
    print(f"SINMOD crs: {crs_sinmod}")
    return crs_sinmod

crs_sinmod = obtain_sinmod_crs(...)


#next, transform all coral points to SINMOD grid coordinates

def transform_coral_coordinates(coral_data, crs_sinmod):
    
    #define the crs for the coral data
    crs_wgs84 = CRS.from_epsg(4326)  #WGS84 (lat/lon coordinates)

    transformer_wgs84_to_sinmod = Transformer.from_crs(crs_wgs84, crs_sinmod, always_xy=True)

    #transform coral coordinates from WGS84 to SINMOD
    coral_x, coral_y = transformer_wgs84_to_sinmod.transform(coral_data['lengdegrad'], coral_data['breddegrad'])

    #save coral_x and coral_y as new columns
    coral_data.loc[:, 'x'] = coral_x
    coral_data.loc[:, 'y'] = coral_y

    return coral_data

transformed_coral_data = transform_coral_coordinates(gdf_all_coral_files, crs_sinmod)


In [ ]:
#Remove coral points outside model area

def remove_coral_points_outside_area(transformed_coral_data, sinmod_data):
    xc = sinmod_data.variables['xc'][:]
    yc = sinmod_data.variables['yc'][:]
    xmin, xmax = xc.min(), xc.max()
    ymin, ymax = yc.min(), yc.max()

    coral_x = transformed_coral_data['x']
    coral_y = transformed_coral_data ['y']
    
    valid_indices = ((coral_x >= xmin) & (coral_x <= xmax) & (coral_y >= ymin) & (coral_y <= ymax))

    coral_points_inside_area = transformed_coral_data[valid_indices].copy()
    
return coral_points_inside_area

coral_data = remove_coral_points_outside_area(transformed_coral_data, sinmod_data)

In [ ]:
#save to your chosen directory
output_directory = '/coral-mapping/processed_data/'
output_file = 'coral_data_processed.parquet' 
os.makedirs(output_directory, exist_ok=True)

coral_data.to_parquet(os.path.join(output_directory, output_file), index=False)

In [ ]:
#How to load coral data later:
#coral_filepath = '/output_directory/output_file'
#coral_data = pd.read_parquet(coral_filepath)

In [ ]:
#OPTIONAL
#plot all coral points on top of bathymetry to view their position in model area

bathymetry = sinmod.variables['bathymetry'] #[000]?

plt.figure(figsize=(10, 8))
plt.pcolormesh(sinmod.variables['xc'][:], sinmod.variables['yc'][:], bathymetry, cmap='viridis', shading='auto')
scatter = plt.scatter(coral_data['x'], coral_data['y'], color='red', label='Coral points in model area', s=10)
plt.xlabel("xc (SINMOD grid coordinates)")
plt.ylabel("yc (SINMOD grid coordinates)")
plt.legend()
plt.show()